In [1]:
# Get the data for expeirment
import sklearn
from sklearn.datasets import fetch_california_housing
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
#data['Station Name'].unique()

In [3]:
station_names = {
    'Edmonton Blatchford' : 'A_',
    "Edmonton Int'L CS" : 'B_',
    'Edmonton Namao AWOS A': 'C_',
    'Oliver AGDM' : 'D_',
    'St. Albert Research' : 'E_',
    'Edmonton South Campus UA' : 'Test_'
}

col_to_use = {
     'Date (Local Standard Time)' : 'Date',
     'Air Temp. Inst. (°C)' : 'Air_temp',
     'Precip. (mm)' : 'Precip',
     'Wind Speed 10 m Syno. (km/h)' : 'Syno_wind_speed',
     'Wind Dir. 10 m Syno. (°)' : 'Syno_wind_dir',
     'Wind Speed 10 m Avg. (km/h)' : 'Avg_wind_speed',
     'Wind Dir. 10 m Avg. (°)' : 'Avg_wind_dir'
}

col_to_drop = [
    'Air Temp. Inst. Source Flag',
    'Air Temp. Inst. Comment',
    'Precip. Source Flag',
    'Precip. Comment',
     'Wind Speed 10 m Syno. Source Flag',
     'Wind Speed 10 m Syno. Comment',
     'Wind Dir. 10 m Syno. Source Flag',
     'Wind Dir. 10 m Syno. Comment',
     'Wind Speed 10 m Avg. Source Flag',
     'Wind Speed 10 m Avg. Comment',
     'Wind Dir. 10 m Avg. Source Flag',
     'Wind Dir. 10 m Avg. Comment'
]

In [4]:
# Create new table headers
generated_table_headers = []

for station_name_key, station_name_value in station_names.items():
    for col_name, col_value in col_to_use.items():
        if col_name != 'Date (Local Standard Time)':
            generated_table_headers.append(f'{station_name_value}{col_value}')
        
# Insert the date as the first column 
generated_table_headers.insert(0, 'Date')

In [5]:
def process_raw_data(raw_data, target_df):
    for i in range(1,len(raw_data)): 
        current_row = raw_data.iloc[i]
        current_station_name = current_row['Station Name']
        new_row = {
           'Date'  :  current_row['Date (Local Standard Time)']
        }
                
        for col_name, col_value in col_to_use.items(): 
            if col_name != 'Date (Local Standard Time)':
                new_row[f"{station_names[current_station_name]}{col_value}"] = current_row[col_name]
            
        new_row = pd.DataFrame(new_row, index=[0])
        target_df = pd.concat([new_row,target_df.loc[:]]).reset_index(drop=True)
        
    return target_df

In [6]:
#processed_df = pd.DataFrame(columns = generated_table_headers)

In [7]:
def post_process_and_save_df(target_df, file_name):
    target_df = target_df.groupby(['Date']).first().reset_index()
    target_df['Date'] = pd.to_datetime(target_df['Date'])
    target_df.set_index('Date', inplace=True)
    target_df = target_df.sort_index()
    target_df = target_df.reindex(columns = generated_table_headers)
    target_df = target_df.drop(columns=['Date'])
    #Save
    target_df.to_csv(f'Data\{file_name}.txt', sep = ',')

In [9]:
# Process the data
csv_names = [
    'ACISHourlyData-20170101-20170701',
]

for csv_name in csv_names:
    data = pd.read_csv(f'Data\{csv_name}.csv', encoding = "ISO-8859-1")
    processed_df = pd.DataFrame(columns = generated_table_headers)
    processed_df = process_raw_data(data, processed_df)
    post_process_and_save_df(processed_df, f'processed_wind_small')
